In [1]:
import os
import sys
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import random
import gc
import pandas as pd
from collections import defaultdict


import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torch.nn.utils import weight_norm

# import torch.distributed as dist

# import torchopt
# import functorch

import higher

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
device

device(type='cuda')

In [14]:
import typing as _typing
# isinstance(optim.Adam, optim.Optimizer)
isinstance(optim.Adam(),torch.optim.Optimizer)

False

# __*Parameters*__

In [2]:
world_size = torch.cuda.device_count() ## Number of GPU's available

num_classes = 10                       ## Number of classes in the dataset
inner_loop_iterations = 32             ## Number of iterations in the inner training loop

noise_size = 128                       ## Dimension of the noise vector used by the Generator
batch_size = 256                       ## Number of images in a batch in the inner loop

use_curriculum = True                  ## Enables Curriculum Learning

world_size

1

# __*Importing Data*__

In [3]:
transform = transforms.Compose([
    transforms.Lambda(lambda x: np.array(x)),
    transforms.ToTensor(),
#     torchvision.transforms.Resize((32,48),antialias=False),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])

# train = datasets.MNIST('../data', train=True, transform=transform, download=True)
# train, val = torch.utils.data.random_split(train, [50000, 10000])
# test = datasets.MNIST('../data', train=False, transform=transform, download=True)


train = datasets.CIFAR10('../data', train=True, transform=transform, download=True)
train, val = torch.utils.data.random_split(train, [inner_loop_iterations * batch_size, 50000 - inner_loop_iterations * batch_size])
test = datasets.CIFAR10('../data', train=False, transform=transform, download=True)

train = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle=True, drop_last=True)
val = torch.utils.data.DataLoader(val, batch_size = batch_size * 2, shuffle=True, drop_last=True)
test = torch.utils.data.DataLoader(test, batch_size = batch_size * 2, shuffle=True, drop_last=True)

steps_per_epoch = len(val)

Files already downloaded and verified
Files already downloaded and verified


In [4]:

import torchmetrics
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall

collection = torchmetrics.MetricCollection([
    MulticlassAccuracy(num_classes=10, average="micro"),
    MulticlassPrecision(num_classes=10, average="macro"),
    MulticlassRecall(num_classes=10, average="macro")
])

c:\Users\Raunaq\Desktop\GTNCode\env\gtn\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [5]:

# collection = torchmetrics.MetricCollection(
#     torchmetrics.Accuracy(task="multiclass" , num_classes = num_classes),
# #     torchmetrics.Recall(task="multiclass" , num_classes = num_classes),
# #     torchmetrics.Precision(task="multiclass" , num_classes = num_classes),
# )
# collection = torchmetrics.Accuracy(task="multiclass" , num_classes = num_classes)

In [6]:
%load_ext autoreload
%autoreload 2
import virtusagtn
import importlib
importlib.reload(virtusagtn) 
import virtusagtn.Models as Models
import torchmetrics

mylearners = [Models.Learner(img_size=[3,32,32], num_classes=10,
                cnn_filters = np.random.randint(low=32, high=256, size=(3,)),
                linear_filters = np.random.randint(low=32, high=128, size=(1,)))
                for i in range(1)]
cnngtn = virtusagtn.GTN.DataGTN(loss_fn = torch.nn.CrossEntropyLoss(),
                        learnerlist=mylearners,
                        num_classes=10, 
                      batch_size = 4,
                      plot_steps = 25,
                      metrics = collection,
                        device = device)
cnngtn.compile(curriculum_loader = train)

cnngtn.compileoptimizer(inner_opt=torch.optim.SGD , 
  inner_opt_params = {'lr':0.01} , 
  override_params = {'lr':0.02, 'momentum':0.9},
  outer_opt = torch.optim.Adam, 
  outer_opt_params = {'lr':0.01, 'betas':(0.9,0.9)})

In [7]:
df = cnngtn.train(val, test, path="gtn",epochs=1)

{'MulticlassAccuracy': tensor(0.1650, device='cuda:0'), 'MulticlassPrecision': tensor(0.1622, device='cuda:0'), 'MulticlassRecall': tensor(0.1610, device='cuda:0')}
{'MulticlassAccuracy': tensor(0.2773, device='cuda:0'), 'MulticlassPrecision': tensor(0.2599, device='cuda:0'), 'MulticlassRecall': tensor(0.2777, device='cuda:0')}
{'MulticlassAccuracy': tensor(0.2480, device='cuda:0'), 'MulticlassPrecision': tensor(0.2500, device='cuda:0'), 'MulticlassRecall': tensor(0.2523, device='cuda:0')}
{'MulticlassAccuracy': tensor(0.3018, device='cuda:0'), 'MulticlassPrecision': tensor(0.2961, device='cuda:0'), 'MulticlassRecall': tensor(0.3046, device='cuda:0')}
{'MulticlassAccuracy': tensor(0.3574, device='cuda:0'), 'MulticlassPrecision': tensor(0.3459, device='cuda:0'), 'MulticlassRecall': tensor(0.3605, device='cuda:0')}
{'MulticlassAccuracy': tensor(0.3613, device='cuda:0'), 'MulticlassPrecision': tensor(0.3563, device='cuda:0'), 'MulticlassRecall': tensor(0.3627, device='cuda:0')}
{'Multicla

In [8]:
df.head()

,InnerMulticlassAccuracy,InnerMulticlassPrecision,InnerMulticlassRecall,InnerLoss,TrainMulticlassAccuracy,TrainMulticlassPrecision,TrainMulticlassRecall,TrainLoss,TestMulticlassAccuracy,TestMulticlassPrecision,TestMulticlassRecall,TestLoss,Path
0,"[0.165, 0.302, 0.359, 0.43, 0.473, 0.424, 0.49...","[0.162, 0.296, 0.359, 0.421, 0.475, 0.421, 0.4...","[0.161, 0.305, 0.358, 0.424, 0.471, 0.427, 0.4...","[2.172, 1.808, 1.723, 1.527, 1.515, 1.504, 1.5...","[0.277, 0.357, 0.428, 0.434, 0.447, 0.482, 0.5...","[0.26, 0.346, 0.429, 0.44, 0.442, 0.477, 0.537...","[0.278, 0.361, 0.429, 0.439, 0.451, 0.476, 0.5...","[2.067, 1.812, 1.601, 1.582, 1.522, 1.477, 1.3...","[0.248, 0.361, 0.387, 0.486, 0.473, 0.486, 0.4...","[0.25, 0.356, 0.386, 0.492, 0.466, 0.486, 0.47...","[0.252, 0.363, 0.384, 0.484, 0.464, 0.484, 0.4...","[2.123, 1.761, 1.721, 1.516, 1.474, 1.408, 1.3...",gtn/0.pth


# __*Initialization*__

In [9]:
# use_teacher = False
# use_curriculum = True 

# params_to_train = []

# if use_teacher == True:
#     teacher = Teacher().to(device)
# #     teacher = nn.DataParallel(teacher, device_ids=list(range(torch.cuda.device_count())))
    
#     params_to_train += list(teacher.parameters())
#     if use_curriculum ==True:
#         teacher_noise = nn.Parameter(torch.randn(inner_loop_iterations, batch_size, noise_size), requires_grad=True) 
#         params_to_train += [teacher_noise]
    
#     teacher_labels = torch.arange(batch_size) % num_classes                        ## Creating Labels for Generator
#     one_hot = nn.functional.one_hot(teacher_labels, num_classes)                                        ## One hot encoding above labels
        
# else: 
#     curriculum_data=[]
#     curriculum_labels = []

#     for i in range(inner_loop_iterations):
#         loader = next(train_loader)
#         curriculum_data  +=[loader[0]]
#         curriculum_labels+=[loader[1]]

#     curriculum_data = nn.Parameter(torch.stack((curriculum_data),0).detach(),requires_grad=True)
#     curriculum_labels = torch.stack(curriculum_labels,0).detach()
#     params_to_train += [curriculum_data]

# optimizer_name = "sgd"
# optimizer_input = {'lr': 0.02, 'momentum':0.9}
# # optimizer_input = {'lr': 0.02}

    
# if optimizer_name == 'adam':
# #     ADAM: lr, weight_decay, amsgrad, betas
#     default_optimizer = optim.Adam
    
# if optimizer_name == 'sgd':
#     #  SGD: lr, weight_decay, momentum, dampening, nesterov
#     default_optimizer = optim.SGD

# optim_params = [nn.Parameter(Tensor(x).to(device)) if isinstance(x,list) else nn.Parameter(Tensor([x]).to(device))  for x in optimizer_input.values()]     
# # optim_params = nn.Parameter(Tensor(list(optimizer_input.values())).to(device), requires_grad=True)                          ## Parameters for SGD optimizer are also learnt 
# params_to_train += optim_params


# optimizer_teacher = optim.Adam(params_to_train,0.002, [0.9,0.9])                                                           ## Optimizer for Generator
# loss_fn = nn.CrossEntropyLoss()                                                                                ## Loss Fn used by the Learner

# epochs = 1
# plot_steps = 25
# batch_exe = 4         
# def divide_chunks(l, n):
#     for i in range(0, len(l), n): 
#         yield l[i:i + n]
  
# batches = list(divide_chunks(list(range(inner_loop_iterations)), batch_exe))

# path='./checkpoints'
# if not os.path.exists(path):
#     os.makedirs(path)

# __*GTN Training Loop*__

In [ ]:
# if use_teacher == True:
#     curriculum_data=[]
#     curriculum_labels=[]
#     with torch.no_grad():
#         for step in range(inner_loop_iterations):
#             curriculum_data += [teacher(teacher_noise[step].to(device),one_hot.to(device)).detach()]
#             curriculum_labels += [teacher_labels]
#             curriculum_data[step].requires_grads = False
#             plot_results(curriculum_data[step])
            
#         curriculum_data = nn.Parameter(torch.stack((curriculum_data),0).detach(),requires_grad=False)

# __*Results*__

In [ ]:
# plt.style.use('ggplot')
# plt.rcParams["figure.figsize"] = (16,4)


# plt.subplot(1,2,1)
# plt.plot(range(LIM),gtn["Train Loss"].apply(lambda x:x[-1]),              label="GTN",              linewidth=0.75, color="black")
# plt.plot(range(LIM),normaldataset["Train Loss"].apply(lambda x:x[-1]),    label="Normal Dataset",   linewidth=0.75, color="red")
# plt.plot(range(LIM),normalcurriculum["Train Loss"].apply(lambda x:x[-1]), label="Normal Curriculum",linewidth=0.75, color="green")
# # plt.plot(range(LIM),gtncurriculum["Train Loss"].apply(lambda x:x[-1]),    label="GTN Curriculum",   linewidth=0.75, color="blue")
# # plt.plot(range(LIM),gtndataset["Train Loss"].apply(lambda x:x[-1]),       label="GTN Dataset",      linewidth=0.75, color="gray")
# plt.title("Train Loss")
# plt.legend()
# plt.subplot(1,2,2)
# plt.plot(range(LIM),gtn["Train Accuracy"].apply(lambda x:x[-1]),              label= "GTN",             linewidth=0.75, color= "black")
# plt.plot(range(LIM),normaldataset["Train Accuracy"].apply(lambda x:x[-1]),    label="Normal Dataset",   linewidth=0.75, color= "red")
# plt.plot(range(LIM),normalcurriculum["Train Accuracy"].apply(lambda x:x[-1]), label="Normal Curriculum",linewidth=0.75, color= "green")
# # plt.plot(range(LIM),gtncurriculum["Train Accuracy"].apply(lambda x:x[-1]),    label="GTN Curriculum",   linewidth=0.75, color= "blue")
# # plt.plot(range(LIM),gtndataset["Train Accuracy"].apply(lambda x:x[-1]),       label="GTN Dataset",      linewidth=0.75, color= "gray")

# plt.title("Train Accuracy")
# plt.legend()
# plt.show()

# plt.subplot(1,2,1)
# plt.plot(range(LIM),gtn["Validation Loss"].apply(lambda x:x[-1]),              label="GTN",              linewidth=0.75, color="black")
# plt.plot(range(LIM),normaldataset["Validation Loss"].apply(lambda x:x[-1]),    label="Normal Dataset",   linewidth=0.75, color="red")
# plt.plot(range(LIM),normalcurriculum["Validation Loss"].apply(lambda x:x[-1]), label="Normal Curriculum",linewidth=0.75, color="green")
# # plt.plot(range(LIM),gtncurriculum["Validation Loss"].apply(lambda x:x[-1]),    label="GTN Curriculum",   linewidth=0.75, color="blue")
# # plt.plot(range(LIM),gtndataset["Validation Loss"].apply(lambda x:x[-1]),       label="GTN Dataset",      linewidth=0.75, color="gray")
# plt.title("Validation Loss")
# plt.legend()
# plt.subplot(1,2,2)
# plt.plot(range(LIM),gtn["Validation Accuracy"].apply(lambda x:x[-1]),              label= "GTN",             linewidth=0.75, color= "black")
# plt.plot(range(LIM),normaldataset["Validation Accuracy"].apply(lambda x:x[-1]),    label="Normal Dataset",   linewidth=0.75, color= "red")
# plt.plot(range(LIM),normalcurriculum["Validation Accuracy"].apply(lambda x:x[-1]), label="Normal Curriculum",linewidth=0.75, color= "green")
# # plt.plot(range(LIM),gtncurriculum["Validation Accuracy"].apply(lambda x:x[-1]),    label="GTN Curriculum",   linewidth=0.75, color= "blue")
# # plt.plot(range(LIM),gtndataset["Validation Accuracy"].apply(lambda x:x[-1]),       label="GTN Dataset",      linewidth=0.75, color= "gray")
# plt.title("Validation Accuracy")
# plt.legend()
# plt.show()

# plt.subplot(1,2,1)
# plt.plot(range(LIM),gtn["Test Loss"].apply(lambda x:x[-1]),              label="GTN",              linewidth=0.75, color="black")
# plt.plot(range(LIM),normaldataset["Test Loss"].apply(lambda x:x[-1]),    label="Normal Dataset",   linewidth=0.75, color="red")
# plt.plot(range(LIM),normalcurriculum["Test Loss"].apply(lambda x:x[-1]), label="Normal Curriculum",linewidth=0.75, color="green")
# # plt.plot(range(LIM),gtncurriculum["Test Loss"].apply(lambda x:x[-1]),    label="GTN Curriculum",   linewidth=0.75, color="blue")
# # plt.plot(range(LIM),gtndataset["Test Loss"].apply(lambda x:x[-1]),       label="GTN Dataset",      linewidth=0.75, color="gray")
# plt.title("Test Loss")
# plt.legend()
# plt.subplot(1,2,2)
# plt.plot(range(LIM),gtn["Test Accuracy"].apply(lambda x:x[-1]),              label= "GTN",             linewidth=0.75, color= "black")
# plt.plot(range(LIM),normaldataset["Test Accuracy"].apply(lambda x:x[-1]),    label="Normal Dataset",   linewidth=0.75, color= "red")
# plt.plot(range(LIM),normalcurriculum["Test Accuracy"].apply(lambda x:x[-1]), label="Normal Curriculum",linewidth=0.75, color= "green")
# # plt.plot(range(LIM),gtncurriculum["Test Accuracy"].apply(lambda x:x[-1]),    label="GTN Curriculum",   linewidth=0.75, color= "blue")
# # plt.plot(range(LIM),gtndataset["Test Accuracy"].apply(lambda x:x[-1]),       label="GTN Dataset",      linewidth=0.75, color= "gray")
# plt.title("Test Accuracy")
# plt.legend()
# plt.show()

# # plt.tight_layout()